In [1]:
# Re-import necessary libraries after code execution environment reset
import pandas as pd

# Load the uploaded CSV file again
file_path = "C:/Users/Ashiya Garg/Downloads/Grocery_Inventory_and_Sales_Dataset.csv"
df = pd.read_csv(file_path)

# Display the first few rows and column names
df.head(), df.columns


(    Product_ID     Product_Name             Catagory  Supplier_ID  \
 0  29-205-1132       Sushi Rice      Grains & Pulses  38-037-1699   
 1  40-681-9981   Arabica Coffee            Beverages  54-470-2479   
 2  06-955-3428       Black Rice      Grains & Pulses  54-031-2945   
 3  71-594-6552  Long Grain Rice      Grains & Pulses  63-492-7603   
 4  57-437-1828             Plum  Fruits & Vegetables  54-226-4308   
 
   Supplier_Name  Stock_Quantity  Reorder_Level  Reorder_Quantity Unit_Price  \
 0     Jaxnation              22             72                70     $4.50    
 1       Feedmix              45             77                 2    $20.00    
 2        Vinder              30             38                83     $6.00    
 3    Brightbean              12             59                62     $1.50    
 4    Topicstorm              37             30                74     $4.00    
 
   Date_Received Last_Order_Date Expiration_Date      Warehouse_Location  \
 0     8/16/2024    